# Python垃圾回收机制

## 参考文献

* *Python源码剖析*
* *https://www.cnblogs.com/traditional/p/13698244.html*

## 循环引用

Python使用**引用计数**的垃圾回收机制，**但引用计数的垃圾回收机制致命的缺陷，就是无法解决循环引用问题。**

为了解决**循环引用**问题，Python引入了标记-清除和分代回收算法。

用一句描述，Python内存管理计数主要使用引用计数，标记-清除和分代回收是为了解决循环引用问题而引入的补充技术。

循环引用示例：

In [32]:
l1 = []
l2 = []

l1.append(l2)
l2.append(l1)

## 标记-清除（mark-sweep）

无论哪种垃圾回收机制，一般都分为两个阶段：垃圾检测和垃圾回收。

* 垃圾检测，从所有分配的内存中，找出可以回收的内存和不可回收的内存
* 垃圾回收，系统回收垃圾检测的可回收内存

* 寻找根对象（root Object）集合，root Object一般为全局引用和函数栈的引用，这些引用是不能删除的，root Object集合也是垃圾检测的起点
* 从root Object出发，沿着root Object集合的每一个引用，如果能到达对象A，则A是可达的（reachable），可达对象是不能删除的，这个阶段就是垃圾检测阶段
* 当垃圾检测结束后，所有的对象分为可达的对象和不可达的对象，可达的对象将被保留，不可达的对象将被回收，这个阶段称为垃圾回收

三色标记模型：

垃圾回收前，系统中所有的对象和对象间的引用构成一张图，其中对象就是图的节点，引用就是图的边。采用广度优先遍历，所有对象初始白色，沿着引用进行广度优先搜索，发现一个新的对象，标记为灰色，当对象的所有的引用对象都遍历后，该对象标记为黑色。

## 分代垃圾回收

一些列研究表明，无论使用何种语言开发，无论开发何种类型、何种规模的程序，都存在一个共同点，一定比例的内存块生命周几较短，剩余的内存块周期比较长。

标记-清除的垃圾收集所带来的额外操作实际上与系统中总的内存块的数量是相关的，当需要回收的内存越多时，垃圾检测的额外操作就越多。为提高垃圾回收的效率，基于统计规律，采用以空间换时间到策略。

将内存中的对象根据存活时间分为不同的集合，每个集合称为一代，垃圾收集频率随着存活时间增大而减小。也就是说，存活时间越长对象，越不可能是垃圾，应该越少的去收集。

在Python中，分代回收机制一共分3代，一代就是一个链表。

分代垃圾回收的触发机制：

**Python的0代最多可容纳700个container对象，也就是说当0代链表中的对象数超过700后，会触发垃圾回收机制。当第0代会触发10次垃圾回收的时候，会触发第1代的链表垃圾回收。当第1代垃圾回收触发超过10次后，会触发第2代的垃圾回收。一旦触发垃圾回收，`treshold`和`count`会回到初始化状态。**

## 总结

虽然有很多对象挂在垃圾收集的监控链表上，但是很多是引用计数在维护这些对象，只有存在引用计数无能为力的循环引用时，分代回收垃圾回收机制才起作用。

引用计数不为0，存在两种情况：
1. 被程序正在使用的对象
2. 存在循环引用的对象

被程序使用的对象是不能回收的，所以垃圾收集只是处理循环引用的情况。

> Python的垃圾回收就是：引用计数为主，分代回收为辅。

In [36]:
import gc

In [37]:
gc.get_threshold()

(700, 10, 10)